In [1]:
import pandas as pd
import dvc.api
import mlflow
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, accuracy_score, log_loss
import mlflow
import mlflow.sklearn


from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


from urllib.parse import urlparse

from urllib.parse import urlparse

from mlflow.models.signature import infer_signature
from mlflow.utils.environment import _mlflow_conda_env
import numpy as np

from xgboost import XGBClassifier
import pandas as pd
import sys, os

sys.path.append(os.path.abspath(os.path.join('../scripts')))
from Create_models import CreateModel
from CreateModelXGBoost import CreateModelXGBoost


In [2]:
import warnings
warnings.filterwarnings('ignore')

# A/B Testing using Machine learning

## Retrieve Data

In [3]:
path = 'data\AdSmartABdata.csv'
repo = 'https://github.com/SameC137/abtest-mlops'
rev = 'v1'
data_url = dvc.api.get_url(path=path, repo=repo,rev=rev)
if rev=="v1":
    collected_data = pd.read_csv(data_url)
else:
    collected_data = pd.read_csv(data_url,index_col=0)
collected_data

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
...,...,...,...,...,...,...,...,...,...
8072,ffea24ec-cec1-43fb-b1d1-8f93828c2be2,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
8073,ffea3210-2c3e-426f-a77d-0aa72e73b20f,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
8074,ffeaa0f1-1d72-4ba9-afb4-314b3b00a7c7,control,2020-07-04,9,Generic Smartphone,6,Chrome Mobile,0,0
8075,ffeeed62-3f7c-4a6e-8ba7-95d303d40969,exposed,2020-07-05,15,Samsung SM-A515F,6,Samsung Internet,0,0


## Data Processing

### Select only users with a response

Only the users are useful for our anlysis of the project which is whether they grow more aware of the brand

In [4]:
responded=collected_data.loc[(collected_data["yes"]==1) | (collected_data["no"]==1)]



 Remove auction Id

In [5]:
features= responded.drop(["auction_id","no"],axis=1, inplace=False)
features=features.reset_index(drop=True)

Select the Prediction variable 

In [6]:
y=features["yes"]

In [ ]:


# lb = LabelEncoder() 

# features["experiment"]=lb.fit_transform(features["experiment"])

# features["device_make"]=lb.fit_transform(features["device_make"])

# features["browser"]=lb.fit_transform(features["browser"])

# features["date"]=lb.fit_transform(features["date"])

# features.drop("yes",axis=1,inplace=True)

Encoder and Scaler function to scale and  normalize a data frame

In [7]:

def encode_scale_features(df,columns):
    lb=LabelEncoder()
    norm = MinMaxScaler()
    for i in columns:
        df[i]=lb.fit_transform(df[i])   
    norm_fit = norm.fit_transform(df)
    out=pd.DataFrame(norm_fit,columns=df.columns)
    return out

Making Sure the right columns are scaled and encoded for the differnet versions of the data. 

In [8]:
if rev=="v2":
    feat=["experiment","device_make","date"]
else:
    feat=["experiment","device_make","browser","date"]

features.drop("yes",axis=1,inplace=True)
X=encode_scale_features(features,feat)
    

In [8]:
X

,experiment,date,hour,device_make,platform_os,browser
0,1.0,0.285714,0.086957,0.035088,1.0,0.285714
1,1.0,0.142857,0.695652,0.114035,1.0,0.142857
2,1.0,0.428571,0.347826,0.114035,1.0,0.142857
3,0.0,0.714286,0.173913,0.377193,1.0,0.571429
4,0.0,0.000000,0.652174,0.114035,1.0,0.142857
...,...,...,...,...,...,...
1238,1.0,0.285714,0.913043,0.114035,1.0,0.142857
1239,1.0,0.142857,0.043478,0.114035,1.0,0.142857
1240,0.0,0.857143,0.304348,0.114035,1.0,0.142857
1241,0.0,1.000000,0.695652,0.114035,1.0,0.142857


In [ ]:

# X_train, X_test, y_train, y_test = train_test_split(
#    X, y, test_size=0.3, random_state=1)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, y_train, test_size=0.33, random_state=1)


In [9]:
X_Cross, X_test, y_Cross, y_test = train_test_split(
   X, y, test_size=0.1, random_state=1)

## Decision Tree Classifier model 

Create Decision Tree Classifier model 

In [10]:
decision_model=CreateModel(X_train=X_Cross,X_test=X_test,y_train=y_Cross,y_test=y_test,data_version=rev,name="DecisionTreeClassifier",model=DecisionTreeClassifier)

We have our baseline decision tree classifier model below. It has a log loss of 16.57

In [11]:
baseline_decision_model=decision_model.train()

Log loss of model is  16.57879177938149


The feature importances of the different featuers. We can see that experiemnt has a very low importance near the bottom of the features. Menaing it has very little effect on the awareness of the brand

In [12]:

feature_importances = pd.DataFrame((baseline_decision_model.feature_importances_).transpose() , index=X.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

,importance
hour,0.351508
device_make,0.279956
date,0.180833
browser,0.124467
experiment,0.063236
platform_os,0.000000


K fold validation of our model. The details can be seen in the mlflow dashboard. 

In [13]:
decision_model.trainKFold(5)

DecisionTreeClassifier()

Our hyperparameter tuned model. It has a log loss of 16.302 which is lower than our baseline

In [14]:
params = {'criterion': ['gini','entropy'],'max_depth':[4,5,6,7,8,9,10]}
hyperparameter_tuned_decision_model=decision_model.hyperParameterTune(5,search_space=params)

2021/07/26 08:10:58 INFO mlflow.sklearn.utils: Logging the 5 best runs, 9 runs will be omitted.


Log loss of model is  16.30241120364835


The feature importances in our more accurate model. We can see that experiment has gone down to an importance of 0. Which means that it has no effect on awareness of the brand

In [15]:
feature_importances = pd.DataFrame((hyperparameter_tuned_decision_model.feature_importances_).transpose() , index=X.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

,importance
hour,0.551749
device_make,0.325555
browser,0.122696
experiment,0.000000
date,0.000000
platform_os,0.000000


## Xgboost Model

In [16]:
xgboost_model=CreateModelXGBoost(X_train=X_Cross,X_test=X_test,y_train=y_Cross,y_test=y_test,data_version=rev,name="XGBoost",model=XGBClassifier)

We have our xgboost model below. It has a log loss of 16.0.2 which is better than our decision tree model. 

In [17]:
baseline_xgboost=xgboost_model.train()

[08:12:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Log loss of model is  16.026132976386272


The feature importances of the different features in our xgboost baseline model. We can see that the experiment has the biggest importance 

In [21]:
feature_importances = pd.DataFrame((baseline_xgboost.feature_importances_).transpose() , index=X.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

,importance
experiment,0.218254
browser,0.215570
date,0.204279
hour,0.196750
device_make,0.165147
platform_os,0.000000


K fold validation of our model. The details can be seen in the mlflow dashboard. 

In [18]:
xgboost_model.trainKFold(5)

[08:12:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:12:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:12:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:12:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

Our hyperparameter tuned xgboost model has a log loss of 15.473 which improves on our baseline model

In [20]:
search_space = { 'min_child_weight': [ 5,6,7,8,9 ,10],'gamma': [ 5,6,7,8,9],'subsample': [1.0, 1.2, 1.3],'colsample_bytree': [ 0.8, 0.9,1.0],'max_depth': [2,3,4]}

param = {"learning_rate":0.1, "n_estimators":200, "objective":'binary:logistic', "eval_metric":"logloss","silent":True, "nthread":2}
                    
hyperparameter_tuned_xgboost_model=xgboost_model.hyperParameterTune(5,search_space=search_space,params=param)

2021/07/26 08:13:09 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              eval_metric='logloss', gamma=None, gpu_id=None,
              importance_type='gain', interaction_constr...`


[08:14:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




2021/07/26 08:14:41 INFO mlflow.sklearn.utils: Logging the 5 best runs, 805 runs will be omitted.


Log loss of model is  15.47339101525831


Our tuned model puts the experimnet lower than it was before as less important than the hour. The value of its imporance is also low meaning that it does not drive awareness. 

In [30]:
feature_importances = pd.DataFrame((hyperparameter_tuned_xgboost_model.feature_importances_).transpose() , index=X.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

,importance
hour,0.376343
experiment,0.268275
browser,0.227745
device_make,0.127637
date,0.000000
platform_os,0.000000


## Logistic Regression Model

Our Logistic regression model

In [23]:

logistic=CreateModel(X_train=X_Cross,X_test=X_test,y_train=y_Cross,y_test=y_test,data_version=rev,name="LogisticRegression",model=LogisticRegression)

INFO: 'LogisticRegression_Experiment' does not exist. Creating a new experiment


The baseline logistic regression model has a baseline log loss of 15.473. Which makes it better than even our tuned xgboost

In [24]:
baseline_logistic=logistic.train()

Log loss of model is  15.473410205596634


The feature importnace of our features show that our experiment is near the bottom in terms of driving the awareness. 

In [25]:
feature_importances = pd.DataFrame((baseline_logistic.coef_).transpose() , index=X.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

,importance
hour,0.436096
platform_os,0.429683
device_make,0.359812
experiment,0.122306
date,-0.009455
browser,-0.124927


K fold validation of our model. The details can be seen in the mlflow dashboard. 

In [27]:
logistic.trainKFold(5)

LogisticRegression()

The hyper parameter tuned logisit regression model has a log loss of 15.197 which improves on the baseline

In [26]:
search_space = {"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
hyperparameter_tuned_logistic_model=logistic.hyperParameterTune(5,search_space=search_space)

2021/07/26 08:24:26 INFO mlflow.sklearn.utils: Logging the 5 best runs, 9 runs will be omitted.


Log loss of model is  15.197061613760699


The feature imporance of our experiment still remains low meaning that the ad has not improved awareness

In [28]:
feature_importances = pd.DataFrame((baseline_logistic.coef_).transpose() , index=X.columns.tolist(), columns=['importance'])
feature_importances.sort_values('importance', ascending=False)

,importance
hour,0.436096
platform_os,0.429683
device_make,0.359812
experiment,0.122306
date,-0.009455
browser,-0.124927


# Conclusions
We can see from our various models that the experiment does not drive awareness. We must conclude that the exposure to the dynamic Ad by SmartAd has not improved awareness. 

In [ ]:
# logged_model = 'runs:/c352bb377e77478c83d7b4367a1f6ce2/best_estimator'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.sklearn.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# # loaded_model.predict(pd.DataFrame(data))


# feature_importances = pd.DataFrame((loaded_model.coef_).transpose() , index=X.columns.tolist(), columns=['importance'])
# feature_importances.sort_values('importance', ascending=False)

In [ ]:
# #Decision Tree
# logged_model = 'runs:/aaebc2dc3fdc4a2181345f81eebb7ace/best_estimator'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.sklearn.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# # loaded_model.predict(pd.DataFrame(data))


# feature_importances = pd.DataFrame((loaded_model.feature_importances_).transpose() , index=X_train.columns.tolist(), columns=['importance'])
# feature_importances.sort_values('importance', ascending=False)

In [ ]:
# logged_model = 'runs:/514249d44d01476db05cd9819ad12228/best_estimator'

# # Load model as a PyFuncModel.
# loaded_model = mlflow.sklearn.load_model(logged_model)



# feature_importances = pd.DataFrame((loaded_model.feature_importances_).transpose() , index=X_train.columns.tolist(), columns=['importance'])
# feature_importances.sort_values('importance', ascending=False)